In [1]:
%load_ext autoreload
%autoreload 2

from utils.loading import read_saved_file

In [2]:
sequence_df = read_saved_file(type_to_read='Sequence')
taxonomy_df = read_saved_file(type_to_read='Taxonomy')

In [3]:
sequence_df

,reference,sequence,sequence_size,file_num
0,111188,AACGAACGCTGGCGGCATGCCTAACACATGCAAGTCGAACGAGACC...,1408,99
1,111188,GCTGGCGGCGTGCCTAACACATGTAAGTCGAACGGGACTGGGGGCA...,1427,99
2,111188,AGAGTTTGATCATGGCTCAGGATGAACGCTAGCGGCAGGCCTAACA...,1478,99
3,111187,CCTAATGCATGCAAGTCGAACGCAGCAGGCGTGCCTGGCTGCGTGG...,1335,99
4,111187,AGTCGAGCGGTAGCACAGAGAGCTTGCTCTCGGGTGACGAGCGGCG...,1401,99
...,...,...,...,...
386952,377069,CGGTTGATCCTGCCGGTGGTCACTGCTATAAAATTTCGATTAAGAC...,1338,61
386953,382532,AGGTGGCTGCTGTCGGGATGAGATTAAGTCATGCAACTGAGAGCAC...,1462,61
386954,80194,AATTCCCAATAGGTCAGTCAAGTTGGAATGCTGACTGATTTAAAAC...,1351,61
386955,82300,TTCCGGTTGATCCTGCCGGAGGGTACTACTATTCGGATTCGACTAA...,1352,61


In [4]:
taxonomy_df

,reference,file_num,kingdom,phylum,class,order,family,genus,species
0,228054,99,BACTERIA,CYANOBACTERIA,SYNECHOCOCCOPHYCIDEAE,SYNECHOCOCCALES,SYNECHOCOCCACEAE,SYNECHOCOCCUS,NaN
1,228057,99,BACTERIA,PROTEOBACTERIA,ALPHAPROTEOBACTERIA,RICKETTSIALES,PELAGIBACTERACEAE,NaN,NaN
2,73627,99,BACTERIA,ACTINOBACTERIA,ACTINOBACTERIA,ACTINOMYCETALES,MYCOBACTERIACEAE,MYCOBACTERIUM,NaN
3,378462,99,BACTERIA,FIRMICUTES,BACILLI,BACILLALES,STAPHYLOCOCCACEAE,STAPHYLOCOCCUS,NaN
4,89370,99,BACTERIA,FIRMICUTES,BACILLI,BACILLALES,BACILLACEAE,ANOXYBACILLUS,KESTANBOLENS
...,...,...,...,...,...,...,...,...,...
391462,4452949,61,BACTERIA,PROTEOBACTERIA,ALPHAPROTEOBACTERIA,RICKETTSIALES,MITOCHONDRIA,NaN,NaN
391463,4363260,61,BACTERIA,CYANOBACTERIA,CHLOROPLAST,CHLOROPHYTA,NaN,NaN,NaN
391464,3190878,61,BACTERIA,PROTEOBACTERIA,ALPHAPROTEOBACTERIA,RICKETTSIALES,MITOCHONDRIA,NaN,NaN
391465,696036,61,BACTERIA,TENERICUTES,MOLLICUTES,ANAEROPLASMATALES,ANAEROPLASMATACEAE,ASTEROLEPLASMA,NaN
